<a href="https://colab.research.google.com/github/thofes/judging_SJ_BA/blob/main/Mark6_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Connect GDrive
from google.colab import drive
drive.mount('/content.gdrive')

#init Repository - YOLOv5
%cd /content


!git clone https://github.com/thofes/yolov5  # clone
%cd yolov5
%pip install -qr requirements.txt  # install


#init Repository - OpenPose
import os
from os.path import exists, join, basename, splitext

#Change the drive to my mounted gdrive
%cd /content

git_repo_url = 'https://github.com/thofes/openpose'
project_name = splitext(basename(git_repo_url))[0]

if 1 or not exists(project_name):
  !rm -rf openpose
  # see: https://github.com/CMU-Perceptual-Computing-Lab/openpose/issues/949
  print("install new CMake becaue of CUDA10")
  cmake_version = 'cmake-3.20.2-linux-x86_64.tar.gz'
  if not exists(cmake_version):
    !wget -q 'https://cmake.org/files/v3.20/{cmake_version}'
  !tar xfz {cmake_version} --strip-components=1 -C /usr/local

  print("clone openpose")
  !git clone -q --depth 1 $git_repo_url
  print("install system dependencies")
  !apt-get -qq install -y libatlas-base-dev libprotobuf-dev libleveldb-dev libsnappy-dev libhdf5-serial-dev protobuf-compiler libgflags-dev libgoogle-glog-dev liblmdb-dev opencl-headers ocl-icd-opencl-dev libviennacl-dev
  print("build openpose")
  !cd openpose && rm -rf build || true && mkdir build && cd build && cmake .. && make -j`nproc`

In [1]:
#Connect GDrive
from google.colab import drive
drive.mount('/content.gdrive')

Mounted at /content.gdrive


In [28]:
import os
from os.path import exists, join, basename, splitext
import json
import numpy as np

#Aufbau Json File genau anschauen

json_path = "/content.gdrive/My Drive//Train2/data.json"
#json_path = "/content/data.json"
# dictionary to store mapping, labels, and Keypoints
data = {
    "labels":[],
    "keypoints":[]
}

PATH = "/content.gdrive/My Drive/Train2"
label= []
try:
    label = next(os.walk(PATH))[1]
except StopIteration:
    pass
#data["labels"].append(label)
print(label)

for l in label:
  Path_jumpers = PATH + '/' + l
  print(Path_jumpers)
  Jumpers = []
  try:
    Jumpers = next(os.walk(Path_jumpers))[1]
  except StopIteration:
    pass
  print(Jumpers)

  for j in Jumpers:
    Path_Pic = Path_jumpers + '/' + j
    print(Path_Pic)

    %cd /content/yolov5
    !python detect.py --save-crop --source '{Path_Pic}' --project '{Path_Pic}' --save-txt

    folder_dir = Path_Pic + '/exp'
    %cd /content/openpose/build/examples/tutorial_api_python
    !python3 04_keypoints_from_images.py --image_dir '{folder_dir}' --keypoint_scale 3 --no_display true

    with open(Path_Pic + '/exp/keypoints.json', "r") as fp:
        keypoints_json = json.load(fp)

    data["keypoints"].append(keypoints_json)

    if l == 'NoTelemark':
        data["labels"].append(0)
    if l == 'Telemark':
        data["labels"].append(1)    

    #with open(Path_Pic + '/exp/keypoints.txt', "r") as keypointfile:
        #keypoints = keypointfile.read()
    #data["keypoints"].append(keypoints)


with open(json_path, "w") as fp:
  json.dump(data, fp, indent=4)


['NoTelemark', 'Telemark']
/content.gdrive/My Drive/Train2/NoTelemark
['01']
/content.gdrive/My Drive/Train2/NoTelemark/01
NoTelemark
['NoTelemark', 'Telemark']
BP
/content.gdrive/My Drive/Train2/Telemark
['02', '01']
/content.gdrive/My Drive/Train2/Telemark/02
Telemark
['NoTelemark', 'Telemark']
/content.gdrive/My Drive/Train2/Telemark/01
Telemark
['NoTelemark', 'Telemark']


In [50]:

#test load data
import json
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow.keras as keras

with open('/content.gdrive/MyDrive/Train2/data.json', "r") as fp:
    data = json.load(fp)

# convert lists to numpy arrays
X = np.array(data["keypoints"])
#X = np.array(data["mfcc"])
y = np.array(data["labels"])

print(X)
print(X.shape)
print(y)
print(y.shape)


#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)


[[0.61342597 0.30372337 0.23925512 ... 0.52524942 0.71154225 0.54309702]
 [0.4108656  0.33063355 0.24723598 ... 0.4081504  0.68497944 0.33570752]
 [0.70509571 0.25489008 0.19814925 ... 0.59326494 0.7493878  0.5951854 ]]
(3, 1575)
[[0]
 [1]
 [1]]
(3, 1)


In [73]:
import json
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow.keras as keras
import tensorflow as tf

# path to json file that stores MFCCs and genre labels for each processed segment
DATA_PATH = "/content.gdrive/MyDrive/Train2/data.json"
#DATA_PATH = '/content.gdrive/MyDrive/Musik/data_music.json'

def load_data(data_path):
    """Loads training dataset from json file.
        :param data_path (str): Path to json file containing data
        :return X (ndarray): Inputs
        :return y (ndarray): Targets
    """

    with open(data_path, "r") as fp:
        data = json.load(fp)

    # convert lists to numpy arrays
    X = np.array(data["keypoints"])
    #X = np.array(data["mfcc"])
    y = np.array(data["labels"])

    print("Data succesfully loaded!")

    return  X, y


if __name__ == "__main__":

    # load data
    X, y = load_data(DATA_PATH)

    print(X.shape)
    print(y.shape)

    # create train/test split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

    print(X_train)
    print(y_train)
    print(X_test)
    print(y_test)



    # build network topology
    model = keras.Sequential([

        # input layer
        #keras.layers.Flatten(input_shape=(X.shape[0], X.shape[1])),

        # 1st dense layer
        keras.layers.Dense(512, activation='relu', input_shape=(None,1,1575)),

        # 2nd dense layer
        keras.layers.Dense(256, activation='relu'),

        # 3rd dense layer
        keras.layers.Dense(64, activation='relu'),

        # output layer
        keras.layers.Dense(2, activation='sigmoid')
    ])

    # compile model
    optimiser = keras.optimizers.Adam(learning_rate=0.0001)
    model.compile(optimizer=optimiser,
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

    model.summary()

    # train model
    history = model.fit(X_train, y_train, validation_data=(X_test, y_test), batch_size=32, epochs=50)

    

Data succesfully loaded!
(3, 1575)
(3, 1)
[[0.61342597 0.30372337 0.23925512 ... 0.52524942 0.71154225 0.54309702]
 [0.4108656  0.33063355 0.24723598 ... 0.4081504  0.68497944 0.33570752]]
[[0]
 [1]]
[[0.70509571 0.25489008 0.19814925 ... 0.59326494 0.7493878  0.5951854 ]]
[[1]]
Model: "sequential_35"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_145 (Dense)           (None, None, 1, 512)      806912    
                                                                 
 dense_146 (Dense)           (None, None, 1, 256)      131328    
                                                                 
 dense_147 (Dense)           (None, None, 1, 64)       16448     
                                                                 
 dense_148 (Dense)           (None, None, 1, 2)        130       
                                                                 
Total params: 954,818
Trainable param

In [16]:
x3 = np.random.randint(10, size=(3, 1575))  # Three-dimensional array

print(x3)

print("x3 ndim: ", x3.ndim)
print("x3 shape:", x3.shape)
print("x3 size: ", x3.size)

[[7 1 0 ... 4 2 0]
 [0 6 5 ... 8 4 1]
 [3 6 7 ... 6 9 8]]
x3 ndim:  2
x3 shape: (3, 1575)
x3 size:  4725
